antag at hele syntesevejen kan til heme trækkes fra heme_pathway.ipynb
antag at mediet er blevet ændret i media.ipynb

Definér mutanter så de kan bruges i andre filer til karakteriseringer som udbytte af biomasse, heme, osv., osv. :D Gerne som funktioner eller Classes

Opregulering af HEM2, HEM3, HEM13 og HEM 15

Forskellige grader af nedreguleret YFH1 (Binder jern med svovl I mitokondrien. Knockout giver super dårlig vækst. Prøv eventuelt en nedregulering - find en serie af fluxen.)

Knockout af HAP1


<span style="color:red"> Caro note: Jeg tror ikke man kan knocke HAP1 ud, hvor den deltager i regulering (synes det ligner, at det er en aktivator) -> man kan vist kun knocke enzymer ud </span>

In [1]:
from cameo.visualization.plotting.with_plotly import PlotlyPlotter
from cameo.strain_design import OptGene
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite

In [2]:
model = read_sbml_model('iMM904.xml')
model.add_boundary(model.metabolites.pheme_m, type='demand')

Reaction identifier,DM_pheme_m
Name,Protoheme C34H30FeN4O4 demand
Memory address,0x0205368d45e0
Stoichiometry,pheme_m --> Protoheme C34H30FeN4O4 -->
GPR,
Lower bound,0
Upper bound,1000.0


In [3]:
optgene = OptGene(model)

In [8]:
result = optgene.run(target=model.reactions.FCLTm,
                     biomass=model.reactions.BIOMASS_SC5_notrace,
                     substrate=model.metabolites.glc__D_e,
                     max_evaluations=10000,
                     plot=False)

Starting optimization at Tue, 02 Nov 2021 15:34:18


HBox()

Finished after 00:09:17


In [9]:
result

No solutions found


,reactions,genes,size,fva_min,fva_max,target_flux,biomass_flux,yield,fitness


## Import of model and stuff

In [ ]:
#! pip install plotly
# ! pip install bokeh

In [ ]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
import plotly
import bokeh

Name of biomass reaction: BIOMASS_SC5_notrace

# Follow OptGene tutorial 
https://cameo.bio/05-predict-gene-knockout-strategies.html

In [ ]:
from cameo import models
from cameo import phenotypic_phase_plane
from cameo.strain_design import OptGene
from cameo.visualization import plotting

from cameo.visualization.plotting.with_plotly import PlotlyPlotter
#plotter1 = PlotlyPlotter() 

In [ ]:
ecoli_model = models.bigg.iJO1366
wt_solution = ecoli_model.optimize()
growth = wt_solution.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"]
acetate_production = wt_solution.fluxes["EX_ac_e"]

In [ ]:
p = phenotypic_phase_plane(ecoli_model, variables=['BIOMASS_Ec_iJO1366_core_53p95M'], objective='EX_ac_e')

In [ ]:
p.plot(points=[(growth, acetate_production)])

In [ ]:
optgene = OptGene(ecoli_model)

In [ ]:
result = optgene.run(target=ecoli_model.reactions.EX_ac_e,
                     biomass=ecoli_model.reactions.BIOMASS_Ec_iJO1366_core_53p95M,
                     substrate=ecoli_model.metabolites.glc__D_e,
                     max_evaluations=500,
                     plot=False)

In [ ]:
result

In [ ]:
result.plot(plotter)

# _____________________________________________________________________________________

# OptGene with our own model

Documentation for the Cameo OptGene: https://cameo.bio/05-predict-gene-knockout-strategies.html

In [ ]:
#! pip install cameo

In [ ]:
model = read_sbml_model('iMM904.xml')
model.add_boundary(model.metabolites.pheme_m, type='demand')

In [ ]:
optgene = OptGene(model)

In [ ]:
result_own_model = optgene.run(target=model.reactions.DM_pheme_m,
                     biomass=model.reactions.BIOMASS_SC5_notrace,
                     substrate=model.metabolites.glc__D_e,
                     max_evaluations=500,
                     plot=False)

In [ ]:
result_own_model

In [ ]:
result_own_model.plot(0)

# _____________________________________________________________________________________

# Predict expression modulation targets - tutorial
https://cameo.bio/06-predict-gene-modulation-targets.html

In [ ]:
from cameo.strain_design.deterministic.flux_variability_based import FSEOF
from cameo import models

In [ ]:
from cameo.visualization.plotting.with_plotly import PlotlyPlotter
plotter = PlotlyPlotter() 

#Pages where they have the same problem with missing plotter: 
#https://githubmemory.com/repo/biosustain/cameo/issues/283 
#https://groups.google.com/g/cobra-pie/c/BxrVSl_Uwaw?pli=1

In [ ]:
ecoli_model_core = models.bigg.e_coli_core

In [ ]:
fseof = FSEOF(ecoli_model_core)

In [ ]:
fseof.run(target=ecoli_model_core.reactions.EX_succ_e)

In [ ]:
from cameo.flux_analysis.analysis import phenotypic_phase_plane
from cameo.strain_design.deterministic import DifferentialFVA

In [ ]:
production_envelope = phenotypic_phase_plane(ecoli_model_core,
                                             variables=[ecoli_model_core.reactions.BIOMASS_Ecoli_core_w_GAM],
                                             objective=ecoli_model_core.metabolites.succ_e)
production_envelope.plot(plotter,height=400)

In [ ]:
ecoli_model_core.reactions.EX_o2_e.lower_bound = 0
reference_model = ecoli_model_core.copy()
biomass_rxn = reference_model.reactions.BIOMASS_Ecoli_core_w_GAM
biomass_rxn.lower_bound = 0.
target = reference_model.metabolites.succ_e

In [ ]:
diffFVA = DifferentialFVA(design_space_model=ecoli_model_core,
                          reference_model=reference_model,
                          objective=target,
                          variables=[biomass_rxn],
                          normalize_ranges_by=biomass_rxn,
                          points=10)

In [ ]:
result = diffFVA.run(surface_only=True)

In [ ]:
result.solutions

In [ ]:
result.plot(plotter, 5, variables=['FBP', 'G6PDH2r', 'PGL', 'PGK'])

In [ ]:
result.display_on_map(2, map_name="iJO1366.Central metabolism")

In [ ]:
result.display_on_map(3, map_name="iJO1366.Central metabolism")

### Failed OptKnock

documentation:
* https://cobrapy.readthedocs.io/_/downloads/en/0.5.3/pdf/
* https://opencobra.github.io/cobratoolbox/stable/modules/design/index.html?highlight=OptKnock#src.design.OptKnock

In [ ]:
#from cobra import Design
# Note to self: apparantly the design subpackage is not part of the newest version of cobra...

In [ ]:
#all_reactions = []
#for reaction_name in model.reactions:
#    all_reactions.append(reaction_name.id)

In [ ]:
# This cannot be used because the subpackage is not part of this version of cobra

#cobra.design.design_algorithms.set_up_optknock(model, 
#                                               chemical_objective = 'FCLTm', 
#                                               knockable_reactions = all_reactions,
#                                               biomass_objective = 'BIOMASS_SC5_notrace',
#                                               n_knockouts = 5)